In [1]:
import time
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
sample = pd.read_csv('sample.csv')

In [3]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample) #dfの長さ。
ids = list(range(length)) #「0~length-1」の数字の列。

# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break point.
bp1 = length//3 
bp2 = length//3 + bp1

# 順番シャッフルして、Break pointで切り分ける。
random.shuffle(random_ids)
id_A, id_B, id_C = random_ids[:bp1], random_ids[bp1:bp2], random_ids[bp2:]

In [5]:
# それぞれのidを表示。 ランダムにシャッフルされていることがわかる。
print(id_A)
print(id_B)
print(id_C)

[22, 4, 25, 16, 3, 18, 2, 15, 8, 29]
[28, 23, 21, 17, 7, 13, 6, 12, 10, 19]
[20, 1, 11, 27, 30, 9, 26, 14, 24, 5]


In [6]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_A'] = sample['id'].apply(lambda x:x in id_A)
sample['judge_B'] = sample['id'].apply(lambda x:x in id_B)
sample['judge_C'] = sample['id'].apply(lambda x:x in id_C)

In [7]:
# それぞれのidのデータを取得。
df_A = sample[sample['judge_A']].loc[:, ['id','price_A', 'rating_A']]
df_B = sample[sample['judge_B']].loc[:, ['id','price_B', 'rating_B']]
df_C = sample[sample['judge_C']].loc[:, ['id','price_C', 'rating_C']]

In [8]:
# 判別用カラムを削除。
del sample['judge_A']
del sample['judge_B']
del sample['judge_C']

In [9]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
original_price_A, original_point_A = sum(df_A['price_A']), sum(df_A['rating_A'])
original_price_B, original_point_B = sum(df_B['price_B']), sum(df_B['rating_B'])
original_price_C, original_point_C = sum(df_C['price_C']), sum(df_C['rating_C'])

In [10]:
# 獲得した枠のidを貯めていく。
obtained_id_A = []
obtained_id_B = []
obtained_id_C = []

In [11]:
# 獲得した枠の価格を貯めていく。
obtained_price_A = 0
obtained_price_B = 0
obtained_price_C = 0

In [12]:
# 獲得した枠の視聴率を貯めていく。
obtained_point_A = 0
obtained_point_B = 0
obtained_point_C = 0

In [13]:
# ここに結果を格納していき、何度も繰り返す。（視聴率和の合計値が'key'で、その時のidが'value'）
result = dict()

In [14]:
# 各クライアントの考える価値（指定視聴率/値段）を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*1000, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*1000, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*1000, 2)

In [15]:
print('client_Aの交換前の合計価格:', original_price_A, 'client_Aの交換前の合計視聴率', original_point_A)
print('client_Bの交換前の合計価格:', original_price_B, 'client_Bの交換前の合計視聴率', original_point_B)
print('client_Cの交換前の合計価格:', original_price_C, 'client_Cの交換前の合計視聴率', original_point_C)
sample.head(3)

client_Aの交換前の合計価格: 5997 client_Aの交換前の合計視聴率 60.50000000000001
client_Bの交換前の合計価格: 7022 client_Bの交換前の合計視聴率 69.4
client_Cの交換前の合計価格: 6002 client_Cの交換前の合計視聴率 57.8


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,2,943,947,961,10.1,8.9,9.3,10.71,9.40,9.68
2,3,980,980,987,10.4,8.7,10.3,10.61,8.88,10.44


In [16]:
# 交換前のデータをまとめる。
original_prices = [original_price_A, original_price_B, original_price_C]
original_points = [original_point_A, original_point_B, original_point_C]

In [17]:
print(original_prices)
print(original_points)

[5997, 7022, 6002]
[60.50000000000001, 69.4, 57.8]


<b><font color='Red'><font size=4>※valueは、正の数じゃないと、選ばなくなってしまうことに注意。<br>（０の方が大きいから。）</font></font></b>

***
***

### 〜ここからアルゴリズム〜

In [18]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    # 価格が元の価格の1.05%を超えても、終わり。
    if capacity*(1 + rate) - obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        count = sum(np.ones(len(size)) * x)
        # 目的
        objective = cvxpy.Maximize(weight * x / count)
        # 制限。ここでは、capacity(価格)の上下5%を取っている。
        constraints = [capacity*(1 + rate) -  obtained >= size * x]
        constraints += [capacity*(1 - rate) - obtained <= size * x]

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []
    
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

In [19]:
# dfは、上記のsampleのような、カラム名がprice_A, rating_A, value_A, price_B, ...となっているDataFrame。
# namesには、'A', 'B', 'C', ... を入れるイメージ。

def Loop(df, original_prices, original_points, *names):
    result = dict() #結果を格納する。
    num = len(names) #クライアントの数。

    obtained_prices = [0 for i in range(num)] #獲得した枠の値段をそれぞれ格納する。
    obtained_points = [0 for i in range(num)] #獲得した枠の視聴率をそれぞれ格納する。
    obtained_idses = [[] for i in range(num)] #獲得した枠のidをそれぞれ格納する。
    idses = [[] for i in range(num)] # 各ナップザック問題でそれぞれが指定した枠のidを格納する。
    add_idses = [[] for i in range(num)] # 各ナップザック問題の結果、獲得した枠のidを格納する。
    check_point = 0
    
    while check_point==0:
        for i in range(num): #それぞれのクライアントごと。
            original_price = original_prices[i] # 交換前の価格合計。
            original_point = original_points[i] # 交換前の視聴率合計。

            name = names[i] # クライアントの名前。
            size = np.array(df['price_' + name]) # dfのカラム名は、こうなるように指定。全ての枠の値段を取得。
            weight = np.array(df['value_' + name]) # dfのカラム名は、こうなるように指定。全ての枠の価値を取得。
            capacity = original_price 

            if obtained_points[i] < original_point: # 獲得した視聴率和が元々の視聴率和より低かったら
                idses[i] = Knapsack(size, weight, capacity,  obtained_prices[i], rate=0.1)
            else:
                idses[i] = [0 for i in range(len(size))] # 視聴率和が元々の視聴率和を超えていた場合、どのidも選択しない。

        if sum(Flatten_dual(idses)) == 0: #もし、どのクライアントも選んでいなかったら
            idses[0][0] = 1 # 始めのクライアントがdfの一番上の行を選んだことにする。（暫定的なエラー処理。）
            
        ids = np.array(df['id'].index) + 1

        # それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
        ids_dict = dict()
        for key in ids-1: # それぞれのidごとに、
            values = [] # そのidの枠を指定したクライアントを格納するリスト。
            for i in range(num): # それぞれのクライアントごとに、
                if key in idses[i] * ids -1: # そのid（のindex）があれば、
                    values.append(names[i]) # valuesにクライアントの名前が格納される。
            if values: # そのidを指定したクライアントがいれば、
                value = random.choice(values) # その中からクライアントをランダムに選ぶ。
                ids_dict[key] = value # idと合わせて格納される。

        for i in range(num):
            add_idses[i] = [ids for ids, client in ids_dict.items() if client == names[i]] # クライアントごとに、獲得したid
            # obtained_idses[i] += add_idses[i]
            obtained_idses[i] += df.query('index in ' + str(add_idses[i]))['id'].values.tolist()
            obtained_prices[i] += sum(df.query('index in ' + str(add_idses[i]))['price_' + name])
            obtained_points[i] += sum(df.query('index in ' + str(add_idses[i]))['rating_' + name])

        extraction_ids = Flatten_dual(add_idses) # 今回獲得されたidの集合を、取り除く。
        df = df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
        
        if len(df) == 1 and df['id'][0] != 0: #dfの長さが１だとうまく機能しないため、長さが１の場合は加える。
            add_df = pd.DataFrame(np.zeros(df.shape[1])).T # np.zeros()の中の数字は、列の数。
            add_df.columns = df.columns # カラム名を揃える。
            df = pd.concat([df, add_df]).reset_index(drop=True)

        # 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
        if len(df) <= 1:
            check_point += 1
        
        print('loop!!')
        display(df)
    
    if obtained_points > original_points:
        sum_point = sum(obtained_points)
        result[sum_point] = obtained_prices, obtained_idses
                                  
    return result

In [20]:
# 1回のループの結果。
%time
Loop(sample, original_prices, original_points, 'A', 'B', 'C')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
3,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
4,7,757,743,769,7.9,7.6,7.2,10.44,10.23,9.36
5,8,750,738,767,7.0,8.1,7.4,9.33,10.98,9.65
6,9,830,811,827,8.1,8.7,8.1,9.76,10.73,9.79
7,10,357,372,362,3.7,3.3,3.7,10.36,8.87,10.22
8,11,687,703,691,6.3,7.6,6.7,9.17,10.81,9.70
9,12,850,862,848,8.7,8.6,8.2,10.24,9.98,9.67


loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
3,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
4,10,357,372,362,3.7,3.3,3.7,10.36,8.87,10.22
5,13,607,627,608,6.1,7.0,5.1,10.05,11.16,8.39
6,15,230,212,218,2.4,2.0,2.5,10.43,9.43,11.47
7,16,333,323,337,3.0,4.2,2.8,9.01,13.00,8.31
8,18,413,401,429,4.0,4.0,4.4,9.69,9.98,10.26
9,20,417,422,418,3.5,5.9,3.1,8.39,13.98,7.42


loop!!


...

loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,5.0,167.0,149.0,178.0,1.5,1.5,2.0,8.98,10.07,11.24
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00


loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


{}

### これを何度も繰り返すことにより、resultに値を格納しまくる。

In [21]:
%time
results = {}
for i in range(10):
    results.update(Loop(sample, original_prices, original_points, 'A', 'B', 'C'))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
3,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
4,7,757,743,769,7.9,7.6,7.2,10.44,10.23,9.36
5,8,750,738,767,7.0,8.1,7.4,9.33,10.98,9.65
6,9,830,811,827,8.1,8.7,8.1,9.76,10.73,9.79
7,10,357,372,362,3.7,3.3,3.7,10.36,8.87,10.22
8,11,687,703,691,6.3,7.6,6.7,9.17,10.81,9.70
9,12,850,862,848,8.7,8.6,8.2,10.24,9.98,9.67


loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
3,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
4,10,357,372,362,3.7,3.3,3.7,10.36,8.87,10.22
5,13,607,627,608,6.1,7.0,5.1,10.05,11.16,8.39
6,15,230,212,218,2.4,2.0,2.5,10.43,9.43,11.47
7,16,333,323,337,3.0,4.2,2.8,9.01,13.00,8.31
8,18,413,401,429,4.0,4.0,4.4,9.69,9.98,10.26
9,20,417,422,418,3.5,5.9,3.1,8.39,13.98,7.42


loop!!


...

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
1,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
2,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55
3,15,230,212,218,2.4,2.0,2.5,10.43,9.43,11.47


loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,6.0,137.0,155.0,152.0,1.4,1.4,1.3,10.22,9.03,8.55
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00


loop!!


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# 交換の結果。
results

{190.29999999999998: ([6212.0, 6980, 5866],
  [[3, 14, 28, 9, 27, 1, 16, 4, 5, 15, 6.0],
   [2, 21, 8, 19, 24, 26, 13, 20, 10, 18, 22, 23],
   [17, 29, 7, 11, 12, 25, 30]]),
 190.3: ([6393, 6507, 6158],
  [[29, 8, 19, 18, 20, 24, 25, 1, 10, 16, 22, 23],
   [2, 3, 14, 17, 21, 7, 27, 6],
   [28, 9, 11, 12, 26, 13, 30, 4, 5, 15]])}

In [23]:
# 最も視聴率和を大きくした交換の仕方を求める。
print('交換後の価格:',results[max(results.keys())][0])
print('交換後のそれぞれの枠')
print('A:', results[max(results.keys())][1][0])
print('B:', results[max(results.keys())][1][1])
print('C:', results[max(results.keys())][1][2])

交換後の価格: [6393, 6507, 6158]
交換後のそれぞれの枠
A: [29, 8, 19, 18, 20, 24, 25, 1, 10, 16, 22, 23]
B: [2, 3, 14, 17, 21, 7, 27, 6]
C: [28, 9, 11, 12, 26, 13, 30, 4, 5, 15]


<b><font size=4><font color='#113BC5'>この数でも、思いの外時間がかかる…。</font></font></b>